In [17]:
import pandas as pd
import json
import numpy
from pprint import pprint
import requests

In [3]:
# load data using Python JSON module
my_file = '../Share/city_data.json'
with open(my_file ,'r') as f:
    data = json.loads(f.read())
# Flatten data
cities_df = pd.json_normalize(data)


In [4]:
cities_df.columns

Index(['datasetid', 'recordid', 'record_timestamp', 'fields.elevation',
       'fields.name', 'fields.modification_date', 'fields.country',
       'fields.feature_class', 'fields.admin3_code', 'fields.alternate_names',
       'fields.feature_code', 'fields.longitude', 'fields.geoname_id',
       'fields.timezone', 'fields.dem', 'fields.country_code',
       'fields.ascii_name', 'fields.latitude', 'fields.admin1_code',
       'fields.coordinates', 'fields.admin2_code', 'fields.population',
       'geometry.type', 'geometry.coordinates'],
      dtype='object')

In [5]:
#select columns
cities_df = cities_df[['fields.name', 'fields.admin1_code', 'fields.country_code', 'fields.population', 'fields.latitude', 'fields.longitude']]


In [6]:
#rename columns
cities_df = cities_df.rename(columns={'fields.name': 'City', 'fields.admin1_code': 'State', 'fields.country_code': 'Country',
       'fields.population': 'Population', 'fields.latitude': 'Latitude', 'fields.longitude': 'Longitude'})

In [7]:
#create group object containing state groups of cities
city_grp = cities_df.groupby(['State'])

In [8]:
#iterate group object and return max population for each group (state) in group object
lgst_cities = city_grp.apply(lambda g: g[g['Population'] == g['Population'].max()])

In [9]:
#reset the indexes
lgst_cities = lgst_cities.reset_index(drop=True)
lgst_cities

,City,State,Country,Population,Latitude,Longitude
0,Anchorage,AK,US,298695,61.21806,-149.90028
1,Birmingham,AL,US,212461,33.52066,-86.80249
2,Little Rock,AR,US,197992,34.74648,-92.28959
3,Phoenix,AZ,US,1563025,33.44838,-112.07404
4,Los Angeles,CA,US,3971883,34.05223,-118.24368
5,Denver,CO,US,682545,39.73915,-104.9847
6,Bridgeport,CT,US,147629,41.17923,-73.18945
7,Washington,DC,US,601723,38.89511,-77.03637
8,Wilmington,DE,US,71948,39.74595,-75.54659
9,Jacksonville,FL,US,868031,30.33218,-81.65565


In [61]:
#add colums for retrieved data
new_cols = ['Housing', 'Cost of Living', 'Startups', 'Venture Capital', 'Travel Connectivity', 'Commute', 'Business Freedom', 'Safety', 'Healthcare', 'Education', 'Environmental Quality', 'Economy', 'Taxation', 'Internet Access', 'Leisure & Culture']
for col in new_cols:
    lgst_cities[col] = ""
lgst_cities

,City,State,Country,Population,Latitude,Longitude,Housing,Cost of Living,Startups,Venture Capital,...,Commute,Business Freedom,Safety,Healthcare,Education,Environmental Quality,Economy,Taxation,Internet Access,Leisure & Culture
0,Anchorage,AK,US,298695,61.21806,-149.90028,,,,,...,,,,,,,,,,
1,Birmingham,AL,US,212461,33.52066,-86.80249,,,,,...,,,,,,,,,,
2,Little Rock,AR,US,197992,34.74648,-92.28959,,,,,...,,,,,,,,,,
3,Phoenix,AZ,US,1563025,33.44838,-112.07404,,,,,...,,,,,,,,,,
4,Los Angeles,CA,US,3971883,34.05223,-118.24368,,,,,...,,,,,,,,,,
5,Denver,CO,US,682545,39.73915,-104.9847,,,,,...,,,,,,,,,,
6,Bridgeport,CT,US,147629,41.17923,-73.18945,,,,,...,,,,,,,,,,
7,Washington,DC,US,601723,38.89511,-77.03637,,,,,...,,,,,,,,,,
8,Wilmington,DE,US,71948,39.74595,-75.54659,,,,,...,,,,,,,,,,
9,Jacksonville,FL,US,868031,30.33218,-81.65565,,,,,...,,,,,,,,,,


In [99]:
#MAKE REQUEST AND STORE DATA BACK IN DATAFRAME

#The URL is kind of crazy becausae you have to find the city in an urban area and then push the urban area into the query - it's nested.  They provide some instructions, but their example is wrong, so don't follow that.  The instructions are:  "Alternatively, we can find the Urban Area that a city belongs to. E.g. let's find out how the urban area corresponding to Palo Alto, California performs in terms of Teleport scores. Let's use the API Explorer for the city search endpoint to construct the URL. In the search field enter Palo Alto, California, in the embed field, type city:search-results/city:item/city:urban_area/ua:scores, and click the Try it out! button."

#Base URL 
url_open = 'https://api.teleport.org/api/cities/?search='
url_close = "&limit=1&embed=city%3Asearch-results%2Fcity%3Aitem%2Fcity%3Aurban_area%2Fua%3Ascores"

#create a list to loop to gather json data from response
my_categories = pd.DataFrame(new_cols)

print("""
Beginning processing city data:
------------------------------------------""")

#loop dataframe randomly created above and return data / use the index so the return data can be pushed into the existing lgst_cities dataframe
for ind in lgst_cities.index:

    #get city data and transform 
    my_city = lgst_cities['City'][ind].replace(' ', '%20')
    my_state = lgst_cities['State'][ind]

    urban_area = my_city + '%2C%20' + my_state

    # Build query URL
    query_url = f"{url_open}{urban_area}{url_close}"

    #make request and hold response
    response = requests.get(query_url).json()
    
    try:
        print(f"Processing Record {ind + 1} of {len(lgst_cities)}: {my_city}, {my_state}.")
        
        #gather data for each category
        for x in my_categories.index:
                    
            my_category = my_categories.iloc[x, 0]
                    
            #parse json for data
            my_data = response['_embedded']['city:search-results'][0]['_embedded']['city:item']['_embedded']['city:urban_area']['_embedded']['ua:scores']['categories'][x]['score_out_of_10']
                    
            #write the data to the dataframe
            lgst_cities.loc[lgst_cities.index[ind], my_category] = my_data

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



Beginning processing city data:
------------------------------------------
Processing Record 1 of 51: Anchorage, AK.
Processing Record 2 of 51: Birmingham, AL.
Processing Record 3 of 51: Little%20Rock, AR.
Missing field/result... skipping.
Processing Record 4 of 51: Phoenix, AZ.
Processing Record 5 of 51: Los%20Angeles, CA.
Processing Record 6 of 51: Denver, CO.
Processing Record 7 of 51: Bridgeport, CT.
Missing field/result... skipping.
Processing Record 8 of 51: Washington, DC.
Processing Record 9 of 51: Wilmington, DE.
Missing field/result... skipping.
Processing Record 10 of 51: Jacksonville, FL.
Processing Record 11 of 51: Atlanta, GA.
Processing Record 12 of 51: Honolulu, HI.
Processing Record 13 of 51: Des%20Moines, IA.
Processing Record 14 of 51: Boise, ID.
Processing Record 15 of 51: Chicago, IL.
Processing Record 16 of 51: Indianapolis, IN.
Processing Record 17 of 51: Wichita, KS.
Missing field/result... skipping.
Processing Record 18 of 51: Lexington-Fayette, KY.
Missing fi

In [100]:
lgst_cities

,City,State,Country,Population,Latitude,Longitude,Housing,Cost of Living,Startups,Venture Capital,...,Commute,Business Freedom,Safety,Healthcare,Education,Environmental Quality,Economy,Taxation,Internet Access,Leisure & Culture
0,Anchorage,AK,US,298695,61.21806,-149.90028,5.4335,3.141,2.7945,0,...,4.71525,8.671,3.4705,8.63267,3.6245,9.272,6.5145,4.772,4.9645,3.266
1,Birmingham,AL,US,212461,33.52066,-86.80249,6.5555,5.133,4.3575,1,...,2.02075,8.671,1.776,8.74333,3.6245,7.0375,6.5145,4.204,5.1605,5.3835
2,Little Rock,AR,US,197992,34.74648,-92.28959,,,,,...,,,,,,,,,,
3,Phoenix,AZ,US,1563025,33.44838,-112.07404,6.533,6.208,6.5085,4.558,...,4.8285,8.671,2.583,8.56733,4.591,4.62525,6.5145,4.488,6.178,6.81
4,Los Angeles,CA,US,3971883,34.05223,-118.24368,1.5275,4.556,10,10,...,3.677,8.57467,5.705,8.43967,8.6245,4.7315,6.5145,4.7675,5.4965,9.196
5,Denver,CO,US,682545,39.73915,-104.9847,3.8375,5.102,7.8645,6.117,...,4.53,8.671,5.371,8.61567,3.6245,7.11675,6.5145,4.346,5.4185,6.2235
6,Bridgeport,CT,US,147629,41.17923,-73.18945,,,,,...,,,,,,,,,,
7,Washington,DC,US,601723,38.89511,-77.03637,1.2105,3.595,8.774,8.056,...,4.457,8.671,2.1915,8.49067,5.9685,6.99375,6.5145,4.062,3.8255,10
8,Wilmington,DE,US,71948,39.74595,-75.54659,,,,,...,,,,,,,,,,
9,Jacksonville,FL,US,868031,30.33218,-81.65565,6.45,5.359,4.3475,2.545,...,1.40575,8.671,3.8255,8.498,3.6245,7.12025,6.5145,4.772,6.1575,5.6545


In [105]:
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
the_results = requests.get("https://data.cdc.gov/resource/vdpk-qzpr.json?year=2015&state=Minnesota&locality=Metropolitan").json()


# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(the_results)
results_df

,year,cause_of_death,state,state_fips_code,hhs_region,age_range,benchmark,locality,observed_deaths,population,expected_deaths,potentially_excess_deaths,percent_potentially_excess_deaths
0,2015,Cancer,Minnesota,MN,5,0-49,2005 Fixed,Metropolitan,366,2835223,378,9,2.50
1,2015,Cancer,Minnesota,MN,5,0-49,2010 Fixed,Metropolitan,366,2835223,352,17,4.60
2,2015,Cancer,Minnesota,MN,5,0-49,Floating,Metropolitan,366,2835223,306,60,16.40
3,2015,Cancer,Minnesota,MN,5,0-54,2005 Fixed,Metropolitan,683,3142775,695,13,1.90
4,2015,Cancer,Minnesota,MN,5,0-54,2010 Fixed,Metropolitan,683,3142775,678,20,2.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2015,Unintentional Injury,Minnesota,MN,5,0-79,2010 Fixed,Metropolitan,1148,4103581,803,354,30.80
116,2015,Unintentional Injury,Minnesota,MN,5,0-79,Floating,Metropolitan,1148,4103581,912,236,20.60
117,2015,Unintentional Injury,Minnesota,MN,5,0-84,2005 Fixed,Metropolitan,1309,4171817,813,512,39.10
118,2015,Unintentional Injury,Minnesota,MN,5,0-84,2010 Fixed,Metropolitan,1309,4171817,851,459,35.10


In [106]:
results_df['cause_of_death'].value_counts()

Heart Disease                        24
Chronic Lower Respiratory Disease    24
Unintentional Injury                 24
Cancer                               24
Stroke                               24
Name: cause_of_death, dtype: int64